In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install required packages
!pip install tensorflow keras scikit-learn imbalanced-learn torch torchvision pandas numpy matplotlib

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully u

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import tensorflow as tf  # For augmentation and future DL models

# Data directories
base_dir = '/content/drive/MyDrive/data'
train_dir = f'{base_dir}/train'
test_dir = f'{base_dir}/test'
image_size = (128, 128)


In [ ]:
def load_images_from_directory(directory, target_size):
    X, y = [], []
    for label in os.listdir(directory):
        label_path = os.path.join(directory, label)
        if not os.path.isdir(label_path):
            continue
        for file in os.listdir(label_path):
            file_path = os.path.join(label_path, file)
            try:
                img = Image.open(file_path).convert('RGB')
                img = img.resize(target_size)
                X.append(np.array(img))
                y.append(label)
            except Exception as e:
                print(f"Error loading {file_path}: {e}")
    return np.array(X), np.array(y)

X_train, y_train = load_images_from_directory(train_dir, image_size)
X_test, y_test = load_images_from_directory(test_dir, image_size)


In [ ]:
# Flatten images (samples, height*width*channels)
X_train_flat = X_train.reshape(len(X_train), -1)
X_test_flat = X_test.reshape(len(X_test), -1)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flat)
X_test_scaled = scaler.transform(X_test_flat)

# Encode labels
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)


In [ ]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_enc), y=y_train_enc)
class_weight_dict = dict(zip(np.unique(y_train_enc), class_weights))

print("Class weights:", class_weight_dict)


Class weights: {np.int64(0): np.float64(0.915625), np.int64(1): np.float64(1.1015037593984962)}


In [ ]:
clf = SVC(kernel='rbf', C=1, gamma='scale', class_weight=class_weight_dict)
clf.fit(X_train_scaled, y_train_enc)


SVC(C=1,
    class_weight={np.int64(0): np.float64(0.915625),
                  np.int64(1): np.float64(1.1015037593984962)})

In [ ]:
y_pred = clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test_enc, y_pred))
print("Precision:", precision_score(y_test_enc, y_pred, average='weighted'))
print("Recall:", recall_score(y_test_enc, y_pred, average='weighted'))
print("F1 Score:", f1_score(y_test_enc, y_pred, average='weighted'))
print("\nClassification Report:\n", classification_report(y_test_enc, y_pred, target_names=le.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test_enc, y_pred))


Accuracy: 0.8287878787878787
Precision: 0.8390879167578197
Recall: 0.8287878787878787
F1 Score: 0.8289683218078994

Classification Report:
               precision    recall  f1-score   support

      benign       0.90      0.77      0.83       360
   malignant       0.77      0.90      0.83       300

    accuracy                           0.83       660
   macro avg       0.83      0.83      0.83       660
weighted avg       0.84      0.83      0.83       660

Confusion Matrix:
 [[278  82]
 [ 31 269]]
